In [5]:
# Flat direct optimization experiments
basestr = "python directopt.py"

datadirs = ['cyl', 'cylnocut']
inits = ['isom']
# anneals = [True, False]
grads = [None, 'l2', 'split']

f = open("slurm/directopt_tests_v2", 'w')
for data in datadirs:
    for init in inits:
        for grad in grads:
            # Don't anneal if grad
            if anneal and grad is not None: 
                continue 
            expname = f"dopt_v2_data{data}_init{init}_grad{grad}"
            expstr = basestr + f" --vs ./scratch/data/{data}_vs.pt --fs ./scratch/data/{data}_fs.pt --init ./scratch/data/{data}_{init}.pt --savedir ./scratch/{expname}"
            if grad:
                expstr += f" --grad {grad}"
            f.write(expstr + "\n")
f.close() 


In [3]:
# Flat sanity check experiments
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "2000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5",
              "--layer_normalization", "FLATTEN",
              "--lossdistortion", "dirichlet", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)

datadirs = ['cylinder', 'cylinder_nocut']
inits = ['isometric', 'tutte']
sepdeltas = [0.1]
# schedules = [True, False]
grads = [None, 'l2', 'split']

f = open("slurm/cylinder_flat", 'w')
for data in datadirs:
    for delta in sepdeltas:
        for init in inits:
            for grad in grads:
                expname = f"flatcyl_sepd{delta}_data{data}_init{init}_grad{grad}"
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --init {init} --seplossdelta {delta} --expname {expname}"
                if grad:
                    expstr += f" --lossgradientstitching {grad}"
                else: 
                    expstr += f" --lossedgeseparation --seplossweight 1"
                f.write(expstr + "\n")
f.close() 


In [1]:
# Gradient stitching experiments 
basestr = ["--root_dir_train", "./data/cylinder", "--root_dir_test", "./data/cylinder",
             "--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "8000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "0.005",
              "--lossdistortion", "dirichlet", "--lossgradientstitching", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
ffts = [True, False]
stitcheps = [1e-4, 1e-3, 1e-2]
inits = ['isometric', 'tutte']

f = open("slurm/cylinder_grad_exps", 'w')
for fft in ffts: 
    for stitch in stitcheps:
        for init in inits:
            expname = f"cylinder_fft{fft}_eps{stitch}_init{init}"
            expstr = basestr + f" --init {init} --stitcheps {stitch} --expname {expname}"
            if fft:
                expstr += " --fft --fft_dim 256"
            f.write(expstr + "\n")
f.close() 

## No cut cylinder 
basestr = ["--root_dir_train", "./data/cylinder_nocut", "--root_dir_test", "./data/cylinder_nocut",
             "--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "8000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "0.005",
              "--lossdistortion", "dirichlet", "--lossgradientstitching", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
ffts = [True, False]
stitcheps = [1e-4, 1e-3, 1e-2]
inits = ['isometric', 'tutte']

f = open("slurm/cylinder_nocut_grad_exps", 'w')
for fft in ffts: 
    for stitch in stitcheps:
        for init in inits:
            expname = f"cylinder_nocut_fft{fft}_eps{stitch}_init{init}"
            expstr = basestr + f" --init {init} --stitcheps {stitch} --expname {expname}"
            if fft:
                expstr += " --fft --fft_dim 256"
            f.write(expstr + "\n")
f.close() 

In [3]:
basestr = ["--root_dir_train", "./data/cylinder", "--root_dir_test", "./data/cylinder",
             "--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "8000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "0.005",
              "--lossdistortion", "dirichlet", "--lossedgeseparation", "--seplossweight", "1", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1", "--fft", "--fft_dim", "256"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
sepdeltas = [0.01, 0.1, 0.5]
seplosses = ['l1', 'mse']

f = open("slurm/cylinder_exps", 'w')
for delta in sepdeltas:
    for seploss in seplosses:
        expname = f"cylinder_sepd{delta}_sepl{seploss}"
        expstr = basestr + f" --init isometric --eseploss {seploss} --seplossdelta {delta} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

inits = ['isometric', 'tutte']
f = open("slurm/cylinder_exps_v2", 'w')
for init in inits:
    for seploss in seplosses:
        expname = f"cylinder_init{init}_sepl{seploss}"
        expstr = basestr + f" --init {init} --eseploss {seploss} --expname {expname} --seploss_schedule --seplossdelta_min 0.001"
        f.write(expstr + "\n")
f.close() 

# Same for bunny
basestr = ["--root_dir_train", "./data/bunny", "--root_dir_test", "./data/bunny",
             "--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "8000",
             "--val_interval", "20", "--data_file", "bunny.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--init", "isometric", "--lr", "0.005",
              "--lossdistortion", "dirichlet", "--lossedgeseparation", "--seplossweight", "1", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
sepdeltas = [0.01, 0.1, 0.5]
seplosses = ['l1', 'mse']

f = open("slurm/bunny_exps", 'w')
for delta in sepdeltas:
    for seploss in seplosses:
        expname = f"bunny_sepd{delta}_sepl{seploss}"
        expstr = basestr + f" --eseploss {seploss} --seplossdelta {delta} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

In [1]:
argstr = """--root_dir_train ./data/cylinder --root_dir_test ./data/cylinder --expname Debug --experiment_type DEFAULT 
        --size_train 1 --size_test 1 --epochs 5000 --val_interval 20 --data_file cylinder.json --align_2D --xp_type uv 
        --gpu_strategy ddp --n_gpu 1 --no_poisson --lossdistortion dirichlet --identity --init isometric --lossedgeseparation 
        --seplossdelta 0.1 --seplossweight 1 --targets_per_batch 16 --accumulate_grad_batches 1"""
print(" ".join(argstr.split()))

--root_dir_train ./data/cylinder --root_dir_test ./data/cylinder --expname Debug --experiment_type DEFAULT --size_train 1 --size_test 1 --epochs 5000 --val_interval 20 --data_file cylinder.json --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 --no_poisson --lossdistortion dirichlet --identity --init isometric --lossedgeseparation --seplossdelta 0.5 --seplossweight 1 --targets_per_batch 16 --accumulate_grad_batches 1
